<a href="https://colab.research.google.com/github/anilnalluri/GCPSketchnote/blob/main/Targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Targets

In [ ]:
/* ---- Step1 : get required columns ---- */

create or replace table cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans1 as (
select distinct
f.month_id,
m.fy_year_id,
case when kpi_id = 'P90005-T' then 'cs'
     when kpi_id = 'F2001-T' then 'fs'
     when kpi_id = 'P2001-T' then 'ss'
     when kpi_id = 'C2001-T' then 'ds'
	 when kpi_id = 'P90005-T2' then 'ws'
     else 'NA' end as kpi_id,
kpi_name,
case when timeframe = 'Cum Tgt' then 'cumulative'
		 when timeframe = 'Sel Tgt' then 'Monthly'
		 when timeframe = 'Yrly Tgt' then 'Annual' end as time_roll_up,
sap_entity,
client_cd,
currency as currency_ind,
hts,
channel,
numerator_value,
denominator_value,
kpi_value
from cf-mdw-int-mid-dev.metro_load_db_bit.ctargets_load_raw f
JOIN (Select distinct month_id,fy_year_id from metro-bi-dl-prep-mid-pp.metro_dwh_qa.dw_time_month distinct_month) m
	ON (
	f.month_id = m.month_id)
);

---step2: Pivoting and updating KPI columns ---

create or replace table cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans2 as (
with
ct as (
select distinct
month_id,
fy_year_id,
kpi_id,
time_roll_up,
sap_entity,
client_cd,
currency_ind,
hts,
channel,
kpi_value,
from cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans1 -- where hts <> 'Total'
),
com as(
select distinct * from ct
pivot(sum(kpi_value) as k for kpi_id in ('cs', 'fs', 'ds', 'ss', 'ws')))

select distinct
month_id,
fy_year_id,
time_roll_up,
client_cd,
sap_entity,
currency_ind,
hts,
channel,
case when channel = "CC" and hts = "Total" then sum(ifnull(k_cs, 0))
     when channel = "FSD" and hts = "Total" then sum(ifnull(k_fs, 0))
     when channel = "Webshop" and hts = "Total" then sum(ifnull(k_ws, 0)) else 0 end as sales,
sum(ifnull(k_ds, 0)) as Digital_Sales,
case when hts <> "Total" then sum(ifnull(k_ss, 0)) else 0 end as Statistical_Sales
from com
group by 1,2,3,4,5,6,7,8);

--- step3: remove all kpi zero records ---

create or replace table cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans3 as (
select distinct * from  cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans2
where (sales <> 0 or Digital_Sales <> 0 or Statistical_Sales <> 0)
union distinct
select distinct * from  cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans2
where channel = 'Webshop'
);

--- Step4: final temp1 table in dev environment ----

create or replace table cf-mdw-int-mid-dev.metro_load_db_bit.ctarget_final
as
select distinct * from cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans3 where time_roll_up <> 'Annual'
union all
select distinct null as month_id,
fy_year_id,
time_roll_up,
client_cd,
sap_entity,
currency_ind,
hts,
channel,
Sales,
Digital_Sales,
Statistical_Sales
from cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans3 where time_roll_up = 'Annual';

/*
--- updated code need to test

create or replace table cf-mdw-int-mid-dev.metro_load_db_bit.ctarget_final
as
select distinct * from cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans3 where time_roll_up <> 'Annual'
union all
select distinct null as month_id, a.* except (month_id)
from cf-mdw-int-mid-dev.metro_load_db_bit.targets_trans3 a where time_roll_up = 'Annual';

*/
---- CY PY Temp table logic ----

select distinct
a.month_id
a.fy_year_id
a.time_roll_up
a.client_cd
a.sap_entity
a.currency_ind
a.hts
a.channel
a.sales_cy
a.Digital_Sales_cy
a.Statistical_Sales_cy
b.sales_cy
b.Digital_Sales_cy
b.Statistical_Sales_cy
from
(select DISTINCT
month_id
fy_year_id
time_roll_up
client_cd
sap_entity
currency_ind
hts
channel
sales as sales_cy
Digital_Sales as Digital_Sales_cy
Statistical_Sales as Statistical_Sales_cy
from `cf-mdw-int-mid-dev.metro_load_db_bit.ctarget_final`) a
left join (
select DISTINCT
m.month_id
m.fy_year_id
x.time_roll_up
x.client_cd
x.sap_entity
x.currency_ind
x.hts
x.channel
x.sales as sales_py
x.Digital_Sales as Digital_Sales_py
x.Statistical_Sales as Statistical_Sales_py
from `cf-mdw-int-mid-dev.metro_load_db_bit.ctarget_final` x
join (Select Distinct c.month_id, c.fy_year_id, c.last_year_month_id FROM `metro-bi-dl-prep-mid-pp.metro_dwh_devlp.dw_time_month` c) m
On x.month_id = m.last_year_month_id
) b
on a.month_id = b.month_id
a.fy_year_id = b.fy_year_id
a.time_roll_up = b.time_roll_up
a.client_cd = b.client_cd
a.sap_entity = b.sap_entity
a.currency_ind = b.currency_ind
a.hts = b.hts
a.channel = b.channel


---- step5: Final table and view ------------


DROP TABLE IF EXISTS `cf-mdw-int-metro-mid-dev.metro_dwh_dev_msi.dw_cust_targets`;
CREATE OR REPLACE TABLE `cf-mdw-int-metro-mid-dev.metro_dwh_dev_msi.dw_cust_targets`
PARTITION BY RANGE_BUCKET(month_id, GENERATE_ARRAY(0,9999))
CLUSTER BY client_cd, sap_entity
AS
(
SELECT
month_id,
fy_year_id,
time_roll_up,
client_cd,
sap_entity,
currency_ind,
CASE WHEN hts = 'HoReCa' THEN 1
     WHEN hts = 'Trader' THEN 2
     WHEN hts = 'SCO' THEN 3 ELSE 999 END AS cust_hts_id,
hts AS cust_hts_desc,
CASE WHEN channel = 'CC' THEN 1
     WHEN channel = 'FSD' THEN 2
     WHEN channel = 'WEB' THEN 3 ELSE 999 END AS cust_sales_channel_cd,
channel AS cust_sales_channel_desc,
sales AS Sales,
Digital_Sales,
Statistical_Sales
FROM cf-mdw-int-mid-dev.metro_load_db_bit.ctarget_final
WHERE time_roll_up = 'Monthly'
);



DROP VIEW IF EXISTS `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_cust_targets`;
CREATE OR REPLACE VIEW `cf-mdw-int-msi-mid-dev.semantic_msi.dw_v_cust_targets` AS
(SELECT * FROM `cf-mdw-int-metro-mid-dev.metro_dwh_dev_msi.dw_cust_targets`);

